In [1]:
import data.read_data as read
import data.transform_data as transform
from numpy import concatenate


In [2]:
train_csv_file_name = "res/train.csv"
test_csv_file_name = "res/test.csv"
test_label_csv_file_name = "res/test_labels.csv"
submission_csv_file_name = "res/sample_submission.csv"


In [3]:
df_train = read.load_train_csv(train_csv_file_name)


In [4]:
print(df_train)


                      id                                       comment_text  \
0       0000997932d777bf  Explanation\nWhy the edits made under my usern...   
1       000103f0d9cfb60f  D'aww! He matches this background colour I'm s...   
2       000113f07ec002fd  Hey man, I'm really not trying to edit war. It...   
3       0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...   
4       0001d958c54c6e35  You, sir, are my hero. Any chance you remember...   
5       00025465d4725e87  "\n\nCongratulations from me as well, use the ...   
6       0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK   
7       00031b1e95af7921  Your vandalism to the Matt Shirvington article...   
8       00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...   
9       00040093b2687caa  alignment on this subject and which are contra...   
10      0005300084f90edc  "\nFair use rationale for Image:Wonju.jpg\n\nT...   
11      00054a5e18b50dd4  bbq \n\nbe a man and lets 

In [5]:
print(df_train.dtypes)


id               object
comment_text     object
toxic             int64
severe_toxic      int64
obscene           int64
threat            int64
insult            int64
identity_hate     int64
dtype: object


In [6]:
limit_train = -1


In [7]:
X_train = df_train.iloc[:limit_train, :2].values
Y_train = df_train.iloc[:limit_train, 2:].values

X_test = df_train.iloc[limit_train:, :2].values
Y_test = df_train.iloc[limit_train:, 2:].values

X_train = transform.split_comment(X_train)
X_test = transform.split_comment(X_test)

count, vocab = transform.mk_vocab(X_train + X_test)


In [8]:
print(len(vocab))


25385


In [9]:
print(X_train[0])


('0000997932d777bf', array(['explanation', 'why', 'the', 'edits', 'made', 'under', 'my',
       'username', 'hardcore', 'metallica', 'fan', 'were', 'reverted',
       'they', 'weren', 't', 'vandalisms', 'just', 'closure', 'on',
       'some', 'gas', 'after', 'i', 'voted', 'at', 'new', 'york', 'dolls',
       'fac', 'and', 'please', 'don', 't', 'remove', 'the', 'template',
       'from', 'the', 'talk', 'page', 'since', 'i', 'm', 'retired', 'now',
       '89', '205', '38', '27'], dtype='<U11'))


In [10]:
ident_train, X_train = transform.pass_data_to_word_idx(X_train, vocab, count)
# ident_test, X_test = transform.pass_data_to_word_idx(X_test, vocab, count)


In [11]:
max_len = transform.get_max_len_sent(X_train)#transform.get_max_len_sent(concatenate((X_train, X_test), axis=0))
print(max_len)
# print(concatenate((X_train, X_test), axis=0))
print(X_train)


1400
[array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 16,
       34,  3, 35, 36,  3, 37, 38, 39, 24, 40, 41, 42, 43, 44, 45, 46])
 array([47, 48, 49, 50, 51, 52, 53, 24, 40, 54, 55, 56, 57, 37, 58, 59, 60,
       61, 62, 63])
 array([64, 65, 24, 40, 66, 67, 68, 69, 70, 71, 72, 73, 18, 74, 51, 75, 76,
       77, 78, 79, 80, 31, 81, 69, 82, 83,  4, 84, 85,  7, 37, 38, 49, 86,
       69, 87, 88, 89,  3, 90, 91,  3, 92, 93])
 ...
 array([ 133,  104,  105, 7267,   85,  139,   74,   76,  108, 2848,  715,
        133, 1130,   20,    7,   37,   38, 7749, 5812,  451,  214])
 array([ 8954,  2380,   128,    92,   176,   136,  6633,  2059, 15974,
        3645])
 array([  31,   72, 1415,  384,   72,  185,  893,  133,  294, 1130,   20,
          3,  279,   69,  134,    3,  132,  616,  275,   42,   74,   24,
        383,   26,   72])]


In [12]:
X_train = transform.pad_sentences(X_train, max_len, vocab[transform.padding])
# X_test = transform.pad_sentences(X_test, max_len, vocab[transform.padding])


In [13]:
print(X_train[0:10])


[[  1.   2.   3. ...   0.   0.   0.]
 [ 47.  48.  49. ...   0.   0.   0.]
 [ 64.  65.  24. ...   0.   0.   0.]
 ...
 [173. 174.  69. ...   0.   0.   0.]
 [182. 101.   3. ...   0.   0.   0.]
 [218.  20.  51. ...   0.   0.   0.]]


In [14]:
print(X_train.shape)


(159570, 1400)


In [15]:
# print(Y_test[0])
# print(Y_test.shape)


[0 0 0 0 0 0]
(1000, 6)


In [15]:
from model.conv_model import ConvModel
import torch as th
import torch.nn as nn
from utils.cuda import use_cuda
from sklearn.metrics import roc_auc_score


In [16]:
th.cuda.is_available()

True

In [17]:
model = ConvModel(len(vocab), max_len)
loss_fn = nn.BCELoss()

if use_cuda():
    model.cuda()
    loss_fn.cuda()

opt = th.optim.Adagrad(model.parameters(), lr=5e-3)

nb_epoch = 10

batch_size = 32
nb_batch = int(X_train.shape[0] / batch_size)

for e in range(nb_epoch):
    model.train()
    
    if use_cuda():
        model.cuda()
        
    sum_loss = 0
    nb_sent = 0
    
    for i in range(nb_batch):
        i_min = batch_size * i
        i_max = batch_size * (i + 1) if batch_size * (i + 1) < X_train.shape[0] else X_train.shape[0]
                
        x = th.Tensor(X_train[i_min:i_max]).long()
        y = th.Tensor(Y_train[i_min:i_max])
        if use_cuda():
            x = x.cuda()
            y = y.cuda()
        out = model(x)
        loss = loss_fn(out, y)
        loss.backward()
        opt.step()
        
        sum_loss += loss.item()
        nb_sent += 1
    
    sum_loss /= nb_sent
    
    print("Epoch %d : loss = %f" % (e, sum_loss))
    """
    model.eval()
    
    model.cpu()
    
    x_test = th.Tensor(X_test).long()
    y_test = th.Tensor(Y_test)
    res = th.zeros(y_test.size())
    
    batch_size_test = 32
    nb_batch_test = int(x_test.size(0) / batch_size_test)
    
    for i in range(nb_batch_test):
        i_min = batch_size_test * i
        i_max = batch_size_test * (i + 1) if batch_size_test * (i + 1) < x_test.size(0) else x_test.size(0)
        
        x = x_test[i_min:i_max]
        y = y_test[i_min:i_max]
        
        out_test = model(x)
        # diff = th.abs(y - out_test)
        res[i_min:i_max] = out_test
    
    diff = res.mean(dim=0)
    differences = {
        "toxic": diff[0].item(),
        "severe_toxic": diff[1].item(),
        "obscene": diff[2].item(),
        "threat": diff[3].item(),
        "insult": diff[4].item(),
        "identity_hate": diff[5].item()}
    y_test = y_test.detach().numpy()
    res = res.detach().numpy()
    roc_auc_scores = {
        "toxic": roc_auc_score(y_test[:, 0], res[:, 0]),
        "severe_toxic": roc_auc_score(y_test[:, 1], res[:, 1]),
        "obscene": roc_auc_score(y_test[:, 2], res[:, 2]),
        "threat": roc_auc_score(y_test[:, 3], res[:, 3]),
        "insult": roc_auc_score(y_test[:, 4], res[:, 4]),
        "identity_hate": roc_auc_score(y_test[:, 5], res[:, 5])
    }
    
    print(roc_auc_scores)
    print(sum([v for _, v in roc_auc_scores.items()]) / 6)"""


Epoch 0 : loss = 0.086900


Epoch 1 : loss = 0.055634


In [23]:
print(model)


ConvModel(
  (emb): Embedding(11154, 16)
  (seq1): Sequential(
    (0): Conv1d(16, 24, kernel_size=(5,), stride=(1,))
    (1): ReLU()
    (2): Conv1d(24, 32, kernel_size=(3,), stride=(1,))
    (3): ReLU()
    (4): MaxPool1d(kernel_size=1394, stride=1394, padding=0, dilation=1, ceil_mode=False)
  )
  (seq2): Sequential(
    (0): Linear(in_features=32, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=6, bias=True)
    (3): Sigmoid()
  )
)


In [24]:
import pickle


In [25]:
saved_model = model.cpu()
saved_loss = loss_fn.cpu()


In [26]:
pickle.dump(model, open("saved/model.p", "wb"))
pickle.dump(loss_fn, open("saved/loss_fn.p", "wb"))
pickle.dump(opt, open("saved/opt.p", "wb"))
pickle.dump(vocab, open("saved/vocab.p", "wb"))
pickle.dump(count, open("saved/count.p", "wb"))
